In [4]:
import random
import socket
import threading
import hashlib
import time
import datetime
from IPython.display import clear_output
class Cliente_GBN:
    def __init__(self,Ip_server,file,le_wi,le_pa,max_sec,port_send,port_rec):
        self.serverIP=Ip_server # Ip del sevidor 
        self.file = file #Dirección del archivo a abrir 
        self.l_ventana= le_wi # Largo de la ventana
        self.largo_partes = le_pa # Largo de las partes en que se va dividir el archivo
        self.max_secuencia= max_sec # Número de secuencia máximo
        self.port_envio=port_send # Puerto de envío 
        self.port_recibo=port_rec # Puerto de entrada
        self.fill =0 # Largo de los números de secuencias
        self.data =[] # Los paquetes a enviar
        self.vi=0 # Indice inicial de la ventana
        self.vf=le_wi # Indice Final de la ventana
        self.tim= threading.Timer(1,self.enviar_ventana) #Timer para enviar la ventana 
        self.info=[] # info de cada paquete (num_secuencia,num_envios,hora_de_envío,hora_de_rec_ack)
        self.TimeoutInterval=1 # tiempo del timer
        self.EstimateRTT=1
        self.devRTT =0
        # Calcula el checksum de un mensaje en string (sí, es así de simple)
    def calculate_checksum(self,message):
        checksum = hashlib.md5(message.encode()).hexdigest()
        return checksum
    
    # Crear mensaje
    def create_message(self,message, seq_num):
        seq_num_padded = str(seq_num).zfill(self.fill)
        checksum = self.calculate_checksum(message)
        return "%s%s%s" % (str(seq_num_padded), str(checksum), message)
    
    def cargar_datos(self):
        archivo = open(self.file,'r') #Abrir el archivo
        mensaje = archivo.read() # Leer mensaje 
        archivo.close()
        parts = [mensaje[i:i+self.largo_partes] for i in range(0, len(mensaje), self.largo_partes)] # partir el archivo
        self.fill = len(str(self.max_secuencia)) #Sacar el número de caracteres de los numeros de secuencias
        num_secuencia=0 
        for part in parts: #para cada parte del archivo construimos su paquete para enviar.
            if(num_secuencia>self.max_secuencia): # resetear el número de secuencia
                num_secuencia=0
            paquete = self.create_message(part,num_secuencia) #paquete listo para enviar.
            self.data.append(paquete) #agregar paquete a la data
            self.info.append([num_secuencia,0,None,None]) # agregar info del paquete
           
            num_secuencia=num_secuencia+1
        
    # Envía el paquete con datos al servidor
    def send_packet(self,ip, port, message):
        sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
        server_address = (ip, port)
        try:
            sock.sendto(message.encode(), server_address)
        finally:
            sock.close()
            
    def enviar_ventana(self):
           for x in range(self.vi, self.vf+1): #por cada paquete en la ventana
                print("Enviando paquete."+str(x)) 
                info1= self.info[x] #buscamos la info del paquete 
                info1[1] =info1[1]+1 # sumamos un envió del paquete
                info1[2] = datetime.datetime.now() # agregamos la hora d envió
                self.info[x]=info1 # guardamos la info
                self.send_packet(self.serverIP,self.port_envio,self.data[x]) #enviamos
            
    
    def calcEst(self, antEst, sample, desvAnt):
        a = 1/8
        est = (1-a)*antEst+(a*sample)
        b = 1/4
        desv = (1-b)*desvAnt + b*abs(sample-antEst)
        return est, desv

    def receive_ack(self):
       
        sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)

        sock.bind(('127.0.0.1', self.port_recibo))
        sock.settimeout(90)

        while (self.vi!=self.vf):
            data, address = sock.recvfrom(1024) #Esperamos acks
            self.tim.cancel() #Al llegar ack paramos los envios, vamos a ajustar la ventana
            ack = data.decode() 
            num_sec = ack[0:self.fill] # sacamos el número de secuencia.
           
            index=0
            num_sec = int(num_sec) #pasamos a entero el número de secuencia
            for x in range(self.vi, self.vf+1): #buscamos en que indice esta el paquete que llego
                if(self.info[x][0]==num_sec):
                    index=x 
                    break
            print("llego ack de : "+str(index) + " Numsec= "+str(num_sec) )
            self.vi=index #valor inicial de la ventana es el indice que llego.
            pvf= self.vi+self.l_ventana #  posible valor final de la ventana
            if(pvf>=len(self.data)): # el valor final de la ventana no puede ser mayor que la cantidad de datos.
                self.vf=len(self.data)-1
            else:
                self.vf=pvf
            self.info[index][3]= datetime.datetime.now() #Colocamos la hora de llegada
            if(self.info[index][1]==1): #Solo calculamos nuevo tiempo si solo se ha enviado una vez.
                self.info[index][1]=self.info[index][1]+1 #Para no calcular mas de una vez si llegan más acks
                Samplertt = (self.info[index][3]-self.info[index][2]).seconds #Tiempo desde envió hasta recibir acks
                newEst, newdesv= self.calcEst(self.EstimateRTT,Samplertt,self.devRTT) #nuevos valores
                self.EstimateRTT = newEst # setiamos
                self.devRTT=newdesv 
                self.TimeoutInterval = newEst + 4*newdesv
            print("Timer= "+ str(self.TimeoutInterval))
            self.tim= threading.Timer(self.TimeoutInterval,self.enviar_ventana) #volver a iniciar tread.
            self.tim.start()
        self.send_packet(self.serverIP,self.port_envio,"")
        
        
        
    

In [5]:
client = Cliente_GBN("0.0.0.0","divina_comedia.txt",98,900,99,2030,2040)
client.cargar_datos()
print("total de paquetes a enviar"+len(client.data))


621


In [6]:
ack_thread = threading.Thread(target=client.receive_ack)
ack_thread.start()

client.tim.start()


Enviando paquete.0
Enviando paquete.1
Enviando paquete.2
Enviando paquete.3
Enviando paquete.4
Enviando paquete.5
Enviando paquete.6
Enviando paquete.7
Enviando paquete.8
Enviando paquete.9
Enviando paquete.10
Enviando paquete.11
Enviando paquete.12
Enviando paquete.13
Enviando paquete.14
Enviando paquete.15
Enviando paquete.16
Enviando paquete.17
Enviando paquete.18
Enviando paquete.19
Enviando paquete.20
Enviando paquete.21
Enviando paquete.22
Enviando paquete.23
Enviando paquete.24
Enviando paquete.25
Enviando paquete.26
Enviando paquete.27
Enviando paquete.28
Enviando paquete.29
Enviando paquete.30
Enviando paquete.31
Enviando paquete.32
Enviando paquete.33
Enviando paquete.34
Enviando paquete.35
Enviando paquete.36
Enviando paquete.37
Enviando paquete.38
Enviando paquete.39
Enviando paquete.40
Enviando paquete.41
Enviando paquete.42
Enviando paquete.43
Enviando paquete.44
Enviando paquete.45
Enviando paquete.46
Enviando paquete.47
Enviando paquete.48
Enviando paquete.49
Enviando p

llego ack de : 14 Numsec= 14
Timer= 1.875
llego ack de : 14 Numsec= 14
Timer= 1.875
llego ack de : 14 Numsec= 14
Timer= 1.875
llego ack de : 14 Numsec= 14
Timer= 1.875
llego ack de : 14 Numsec= 14
Timer= 1.875
llego ack de : 14 Numsec= 14
Timer= 1.875
llego ack de : 14 Numsec= 14
Timer= 1.875
llego ack de : 14 Numsec= 14
Timer= 1.875
llego ack de : 14 Numsec= 14
Timer= 1.875
llego ack de : 14 Numsec= 14
Timer= 1.875
llego ack de : 14 Numsec= 14
Timer= 1.875
llego ack de : 14 Numsec= 14
Timer= 1.875
llego ack de : 14 Numsec= 14
Timer= 1.875
llego ack de : 14 Numsec= 14
Timer= 1.875
llego ack de : 14 Numsec= 14
Timer= 1.875
llego ack de : 14 Numsec= 14
Timer= 1.875
llego ack de : 14 Numsec= 14
Timer= 1.875
llego ack de : 14 Numsec= 14
Timer= 1.875
llego ack de : 14 Numsec= 14
Timer= 1.875
llego ack de : 14 Numsec= 14
Timer= 1.875
llego ack de : 14 Numsec= 14
Timer= 1.875
llego ack de : 14 Numsec= 14
Timer= 1.875
llego ack de : 14 Numsec= 14
Timer= 1.875
llego ack de : 14 Numsec= 14
Timer

llego ack de : 19 Numsec= 19
Timer= 1.875
llego ack de : 19 Numsec= 19
Timer= 1.875
llego ack de : 19 Numsec= 19
Timer= 1.875
llego ack de : 19 Numsec= 19
Timer= 1.875
llego ack de : 19 Numsec= 19
Timer= 1.875
llego ack de : 19 Numsec= 19
Timer= 1.875
llego ack de : 19 Numsec= 19
Timer= 1.875
llego ack de : 19 Numsec= 19
Timer= 1.875
llego ack de : 19 Numsec= 19
Timer= 1.875
llego ack de : 19 Numsec= 19
Timer= 1.875
llego ack de : 19 Numsec= 19
Timer= 1.875
Enviando paquete.19
Enviando paquete.20
Enviando paquete.21
Enviando paquete.22
Enviando paquete.23
Enviando paquete.24
Enviando paquete.25
Enviando paquete.26
Enviando paquete.27
Enviando paquete.28
Enviando paquete.29
Enviando paquete.30
Enviando paquete.31
Enviando paquete.32
Enviando paquete.33
Enviando paquete.34
Enviando paquete.35
Enviando paquete.36
Enviando paquete.37
Enviando paquete.38
Enviando paquete.39
Enviando paquete.40
Enviando paquete.41
Enviando paquete.42
Enviando paquete.43
Enviando paquete.44
Enviando paquete.4

llego ack de : 23 Numsec= 23
Timer= 1.875
llego ack de : 23 Numsec= 23
Timer= 1.875
llego ack de : 23 Numsec= 23
Timer= 1.875
llego ack de : 23 Numsec= 23
Timer= 1.875
llego ack de : 23 Numsec= 23
Timer= 1.875
llego ack de : 23 Numsec= 23
Timer= 1.875
llego ack de : 23 Numsec= 23
Timer= 1.875
llego ack de : 23 Numsec= 23
Timer= 1.875
llego ack de : 23 Numsec= 23
Timer= 1.875
llego ack de : 23 Numsec= 23
Timer= 1.875
llego ack de : 23 Numsec= 23
Timer= 1.875
llego ack de : 23 Numsec= 23
Timer= 1.875
llego ack de : 23 Numsec= 23
Timer= 1.875
llego ack de : 23 Numsec= 23
Timer= 1.875
llego ack de : 23 Numsec= 23
Timer= 1.875
llego ack de : 23 Numsec= 23
Timer= 1.875
llego ack de : 23 Numsec= 23
Timer= 1.875
llego ack de : 23 Numsec= 23
Timer= 1.875
llego ack de : 23 Numsec= 23
Timer= 1.875
llego ack de : 23 Numsec= 23
Timer= 1.875
llego ack de : 23 Numsec= 23
Timer= 1.875
llego ack de : 23 Numsec= 23
Timer= 1.875
llego ack de : 23 Numsec= 23
Timer= 1.875
llego ack de : 23 Numsec= 23
Timer

llego ack de : 80 Numsec= 80
Timer= 1.875
llego ack de : 80 Numsec= 80
Timer= 1.875
llego ack de : 80 Numsec= 80
Timer= 1.875
llego ack de : 80 Numsec= 80
Timer= 1.875
llego ack de : 80 Numsec= 80
Timer= 1.875
llego ack de : 80 Numsec= 80
Timer= 1.875
llego ack de : 80 Numsec= 80
Timer= 1.875
llego ack de : 80 Numsec= 80
Timer= 1.875
llego ack de : 80 Numsec= 80
Timer= 1.875
llego ack de : 80 Numsec= 80
Timer= 1.875
llego ack de : 80 Numsec= 80
Timer= 1.875
llego ack de : 80 Numsec= 80
Timer= 1.875
llego ack de : 80 Numsec= 80
Timer= 1.875
llego ack de : 80 Numsec= 80
Timer= 1.875
llego ack de : 80 Numsec= 80
Timer= 1.875
llego ack de : 80 Numsec= 80
Timer= 1.875
llego ack de : 80 Numsec= 80
Timer= 1.875
llego ack de : 80 Numsec= 80
Timer= 1.875
llego ack de : 80 Numsec= 80
Timer= 1.875
llego ack de : 80 Numsec= 80
Timer= 1.875
llego ack de : 80 Numsec= 80
Timer= 1.875
Enviando paquete.80
Enviando paquete.81
Enviando paquete.82
Enviando paquete.83
Enviando paquete.84
Enviando paquete.8

...

llego ack de : 351 Numsec= 51
Timer= 1.875
llego ack de : 351 Numsec= 51
Timer= 1.875
llego ack de : 351 Numsec= 51
Timer= 1.875
llego ack de : 351 Numsec= 51
Timer= 1.875
llego ack de : 351 Numsec= 51
Timer= 1.875
llego ack de : 351 Numsec= 51
Timer= 1.875
llego ack de : 351 Numsec= 51
Timer= 1.875
llego ack de : 351 Numsec= 51
Timer= 1.875
llego ack de : 351 Numsec= 51
Timer= 1.875
llego ack de : 351 Numsec= 51
Timer= 1.875
llego ack de : 351 Numsec= 51
Timer= 1.875
llego ack de : 351 Numsec= 51
Timer= 1.875
llego ack de : 351 Numsec= 51
Timer= 1.875
llego ack de : 351 Numsec= 51
Timer= 1.875
llego ack de : 351 Numsec= 51
Timer= 1.875
llego ack de : 351 Numsec= 51
Timer= 1.875
llego ack de : 351 Numsec= 51
Timer= 1.875
llego ack de : 351 Numsec= 51
Timer= 1.875
llego ack de : 351 Numsec= 51
Timer= 1.875
llego ack de : 351 Numsec= 51
Timer= 1.875
llego ack de : 351 Numsec= 51
Timer= 1.875
llego ack de : 351 Numsec= 51
Timer= 1.875
llego ack de : 351 Numsec= 51
Timer= 1.875
llego ack d

llego ack de : 363 Numsec= 63
Timer= 1.875
llego ack de : 364 Numsec= 64
Timer= 1.875
llego ack de : 365 Numsec= 65
Timer= 1.875
llego ack de : 365 Numsec= 65
Timer= 1.875
llego ack de : 365 Numsec= 65
Timer= 1.875
llego ack de : 365 Numsec= 65
Timer= 1.875
llego ack de : 365 Numsec= 65
Timer= 1.875
llego ack de : 365 Numsec= 65
Timer= 1.875
llego ack de : 365 Numsec= 65
Timer= 1.875
llego ack de : 365 Numsec= 65
Timer= 1.875
llego ack de : 365 Numsec= 65
Timer= 1.875
llego ack de : 365 Numsec= 65
Timer= 1.875
llego ack de : 365 Numsec= 65
Timer= 1.875
llego ack de : 365 Numsec= 65
Timer= 1.875
llego ack de : 365 Numsec= 65
Timer= 1.875
llego ack de : 365 Numsec= 65
Timer= 1.875
llego ack de : 365 Numsec= 65
Timer= 1.875
llego ack de : 365 Numsec= 65
Timer= 1.875
llego ack de : 365 Numsec= 65
Timer= 1.875
llego ack de : 365 Numsec= 65
Timer= 1.875
llego ack de : 365 Numsec= 65
Timer= 1.875
llego ack de : 365 Numsec= 65
Timer= 1.875
llego ack de : 365 Numsec= 65
Timer= 1.875
llego ack d

llego ack de : 369 Numsec= 69
Timer= 1.875
llego ack de : 369 Numsec= 69
Timer= 1.875
llego ack de : 369 Numsec= 69
Timer= 1.875
llego ack de : 369 Numsec= 69
Timer= 1.875
llego ack de : 369 Numsec= 69
Timer= 1.875
llego ack de : 369 Numsec= 69
Timer= 1.875
llego ack de : 369 Numsec= 69
Timer= 1.875
llego ack de : 369 Numsec= 69
Timer= 1.875
llego ack de : 369 Numsec= 69
Timer= 1.875
llego ack de : 369 Numsec= 69
Timer= 1.875
llego ack de : 369 Numsec= 69
Timer= 1.875
llego ack de : 369 Numsec= 69
Timer= 1.875
llego ack de : 369 Numsec= 69
Timer= 1.875
llego ack de : 369 Numsec= 69
Timer= 1.875
llego ack de : 369 Numsec= 69
Timer= 1.875
llego ack de : 369 Numsec= 69
Timer= 1.875
llego ack de : 369 Numsec= 69
Timer= 1.875
llego ack de : 369 Numsec= 69
Timer= 1.875
llego ack de : 369 Numsec= 69
Timer= 1.875
llego ack de : 369 Numsec= 69
Timer= 1.875
llego ack de : 369 Numsec= 69
Timer= 1.875
llego ack de : 369 Numsec= 69
Timer= 1.875
llego ack de : 369 Numsec= 69
Timer= 1.875
llego ack d

llego ack de : 371 Numsec= 71
Timer= 1.875
llego ack de : 372 Numsec= 72
Timer= 1.875
llego ack de : 373 Numsec= 73
Timer= 1.875
llego ack de : 374 Numsec= 74
Timer= 1.875
llego ack de : 375 Numsec= 75
Timer= 1.875
llego ack de : 375 Numsec= 75
Timer= 1.875
llego ack de : 375 Numsec= 75
Timer= 1.875
llego ack de : 375 Numsec= 75
Timer= 1.875
llego ack de : 375 Numsec= 75
Timer= 1.875
llego ack de : 375 Numsec= 75
Timer= 1.875
llego ack de : 375 Numsec= 75
Timer= 1.875
llego ack de : 375 Numsec= 75
Timer= 1.875
llego ack de : 375 Numsec= 75
Timer= 1.875
llego ack de : 375 Numsec= 75
Timer= 1.875
llego ack de : 375 Numsec= 75
Timer= 1.875
llego ack de : 375 Numsec= 75
Timer= 1.875
llego ack de : 375 Numsec= 75
Timer= 1.875
llego ack de : 375 Numsec= 75
Timer= 1.875
llego ack de : 375 Numsec= 75
Timer= 1.875
llego ack de : 375 Numsec= 75
Timer= 1.875
llego ack de : 375 Numsec= 75
Timer= 1.875
llego ack de : 375 Numsec= 75
Timer= 1.875
llego ack de : 375 Numsec= 75
Timer= 1.875
llego ack d

llego ack de : 384 Numsec= 84
Timer= 1.875
llego ack de : 384 Numsec= 84
Timer= 1.875
llego ack de : 384 Numsec= 84
Timer= 1.875
llego ack de : 384 Numsec= 84
Timer= 1.875
llego ack de : 384 Numsec= 84
Timer= 1.875
llego ack de : 384 Numsec= 84
Timer= 1.875
llego ack de : 384 Numsec= 84
Timer= 1.875
llego ack de : 384 Numsec= 84
Timer= 1.875
llego ack de : 384 Numsec= 84
Timer= 1.875
llego ack de : 384 Numsec= 84
Timer= 1.875
llego ack de : 384 Numsec= 84
Timer= 1.875
llego ack de : 384 Numsec= 84
Timer= 1.875
llego ack de : 384 Numsec= 84
Timer= 1.875
llego ack de : 384 Numsec= 84
Timer= 1.875
llego ack de : 384 Numsec= 84
Timer= 1.875
llego ack de : 384 Numsec= 84
Timer= 1.875
llego ack de : 384 Numsec= 84
Timer= 1.875
llego ack de : 384 Numsec= 84
Timer= 1.875
llego ack de : 384 Numsec= 84
Timer= 1.875
llego ack de : 384 Numsec= 84
Timer= 1.875
llego ack de : 384 Numsec= 84
Timer= 1.875
llego ack de : 384 Numsec= 84
Timer= 1.875
llego ack de : 384 Numsec= 84
Timer= 1.875
llego ack d

In [7]:
fuser -k 2035/udp


SyntaxError: invalid syntax (<ipython-input-7-a0846c9c7ccc>, line 1)

In [2]:
import datetime

a = datetime.datetime.now()

In [3]:
b = datetime.datetime.now()

In [6]:
print((b-a).seconds)

10


# Envío

In [5]:
# Envía el paquete con datos al servidor
def send_packet(ip, port, message):
    sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
    server_address = (ip, port)

    try:
        sock.sendto(message.encode(), server_address)
    finally:
        sock.close()

# Recibo 

In [6]:
def receive_ack():
    running = True
    sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)

    sock.bind(('0.0.0.0', 2040))
    sock.settimeout(90)

    while running:
        data, address = sock.recvfrom(1024)

        if data:
            print("ACK recibido: " + data.decode())

In [7]:
ack_thread = threading.Thread(target=receive_ack)
ack_thread.start()

seq_num = 0
message = create_message("que pedo perrin", seq_num)
send_packet('0.0.0.0', 2030, message)


hash is 732bfdef3022bcee2a1037438e9689a5
ACK recibido: 00b4b147bc522828731f1a016bfa72c073
